# RLDT

## Step 1: Import the necessary libraries:

In [29]:
import numpy as np
import pandas as pd
import networkx as nx
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

## Step 2: Define the environment:

### Step 2.1: Devices

#### *Gloabl variables*

In [30]:
num_IOT_devices = 5

voltages_frequencies_IOT = [
    (1e6, 1.8),
    (2e6, 2.3),
    (4e6, 2.7),
    (8e6, 4.0),
    (16e6, 5.0),
    (32e6, 6.5),
]
num_MEC_devices = 10

voltages_frequencies_MEC = [
    (6 * 1e8, 0.8),
    (7.5 * 1e8, 0.825),
    (10 * 1e8, 1.0),
    (15 * 1e8, 1.2),
    (30 * 1e8, 2),
    (40 * 1e8, 3.1),
]

task_kinds = [1,2,3,4]

min_num_nodes_dag = 4
max_num_nodes_dag = 20
max_num_parents_dag = 5

num_dag_generations = 100

#### *IOT*

In [31]:
devices_data_IOT = []
for i in range(num_IOT_devices):
    cpu_cores = np.random.choice([4, 6, 8])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            voltages_frequencies_IOT[i]
            for i in np.random.choice(6, size=4, replace=False)
        ],
        "ISL": np.random.randint(10, 21),
        "capacitance": [np.random.uniform(2, 3) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [
            np.random.choice([700, 800, 900]) * 1e-6 for _ in range(cpu_cores)
        ],
        "batteryLevel": np.random.randint(36, 41) * 1e9,
        "errorRate": np.random.randint(1, 6) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.25, 0.75]),
    }
    devices_data_IOT.append(device_info)

IoTdevices = pd.DataFrame(devices_data_IOT)

IoTdevices.set_index("id", inplace=True)
IoTdevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,,,
0,8,"[0, 1, 1, 0, 0, 1, 1, 0]","[(1000000.0, 1.8), (8000000.0, 4.0), (32000000...",11,"[2.0335683668660422e-09, 2.6924918326971967e-0...","[0.0009, 0.0007, 0.0007999999999999999, 0.0009...",3.800000e+10,0.05,"[4, 1]",1
1,4,"[0, 0, 1, 1]","[(16000000.0, 5.0), (2000000.0, 2.3), (3200000...",14,"[2.2224823345252286e-09, 2.5424163291696136e-0...","[0.0007, 0.0007999999999999999, 0.0009, 0.0007]",3.700000e+10,0.03,"[3, 1]",1
2,4,"[0, 0, 0, 0]","[(32000000.0, 6.5), (1000000.0, 1.8), (1600000...",16,"[2.9431546396803817e-09, 2.7712209810637763e-0...","[0.0007999999999999999, 0.0007, 0.000799999999...",4.000000e+10,0.04,"[2, 1, 3]",1
3,8,"[0, 1, 1, 0, 1, 0, 1, 0]","[(32000000.0, 6.5), (16000000.0, 5.0), (100000...",16,"[2.1513162264888043e-09, 2.1126715623024103e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.700000e+10,0.03,"[3, 1, 2]",1
4,6,"[0, 1, 1, 1, 0, 1]","[(1000000.0, 1.8), (2000000.0, 2.3), (32000000...",19,"[2.3115550332130133e-09, 2.938119120978063e-09...","[0.0007, 0.0009, 0.0007999999999999999, 0.0009...",3.800000e+10,0.04,"[1, 2, 3]",1


#### *MEC*

In [32]:
devices_data_MEC = []
for i in range(num_MEC_devices):
    cpu_cores = np.random.choice([16,32,64])
    device_info = {
        "id": i,
        "number_of_cpu_cores": cpu_cores,
        "occupied_cores": [np.random.choice([0, 1]) for _ in range(cpu_cores)],
        "voltages_frequencies": [
            voltages_frequencies_MEC[i]
            for i in np.random.choice(6, size=4, replace=False)
        ],
        "capacitance": [np.random.uniform(1.5, 2) * 1e-9 for _ in range(cpu_cores)],
        "powerIdle": [np.random.choice([9, 9, 10]) * 1e-5 for _ in range(cpu_cores)],
        "errorRate": np.random.randint(5, 11) / 100,
        "accetableTasks": np.random.choice(
            task_kinds, size=np.random.randint(2, 5), replace=False
        ),
        "handleSafeTask": np.random.choice([0, 1], p=[0.75, 0.25]),
    }
    devices_data_MEC.append(device_info)

MECDevices = pd.DataFrame(devices_data_MEC)

MECDevices.set_index("id", inplace=True)
MECDevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,capacitance,powerIdle,errorRate,accetableTasks,handleSafeTask
id,,,,,,,,
0,64,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, ...","[(600000000.0, 0.8), (1500000000.0, 1.2), (750...","[1.819707410427769e-09, 1.6294663562179436e-09...","[9e-05, 0.0001, 9e-05, 0.0001, 9e-05, 9e-05, 9...",0.07,"[3, 4, 1, 2]",0
1,64,"[1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...","[(1000000000.0, 1.0), (4000000000.0, 3.1), (75...","[1.555820637175539e-09, 1.7381429700603459e-09...","[9e-05, 0.0001, 9e-05, 9e-05, 0.0001, 9e-05, 9...",0.08,"[1, 2, 4]",0
2,64,"[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(3000000000.0, 2), (4000000000.0, 3.1), (7500...","[1.8043229015953197e-09, 1.7526913245819908e-0...","[9e-05, 0.0001, 9e-05, 9e-05, 9e-05, 0.0001, 9...",0.05,"[1, 2, 3]",1
3,16,"[1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0]","[(4000000000.0, 3.1), (750000000.0, 0.825), (1...","[1.7246582148524278e-09, 1.5855888829095944e-0...","[0.0001, 9e-05, 9e-05, 9e-05, 9e-05, 9e-05, 9e...",0.09,"[3, 1]",0
4,32,"[0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, ...","[(3000000000.0, 2), (1000000000.0, 1.0), (4000...","[1.765909596360523e-09, 1.5276887016699822e-09...","[0.0001, 9e-05, 9e-05, 9e-05, 0.0001, 9e-05, 9...",0.06,"[2, 1, 4, 3]",0
5,64,"[0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, ...","[(4000000000.0, 3.1), (1000000000.0, 1.0), (30...","[1.7635903262749527e-09, 1.91971944626054e-09,...","[9e-05, 0.0001, 9e-05, 9e-05, 0.0001, 0.0001, ...",0.07,"[1, 2, 3]",0
6,64,"[1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(4000000000.0, 3.1), (600000000.0, 0.8), (100...","[1.6995307914953312e-09, 1.906946589954017e-09...","[9e-05, 9e-05, 9e-05, 9e-05, 0.0001, 0.0001, 9...",0.09,"[3, 2]",1
7,16,"[0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]","[(1000000000.0, 1.0), (600000000.0, 0.8), (300...","[1.8912529355581773e-09, 1.8491490787789526e-0...","[0.0001, 0.0001, 9e-05, 0.0001, 0.0001, 0.0001...",0.09,"[3, 1]",0
8,64,"[0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, ...","[(1500000000.0, 1.2), (4000000000.0, 3.1), (30...","[1.718125167714681e-09, 1.976119544395414e-09,...","[9e-05, 0.0001, 9e-05, 9e-05, 9e-05, 9e-05, 9e...",0.08,"[4, 3, 1, 2]",0


#### *CLOUD*

In [33]:
cloud = [(2.8e9, 1), (3.9e9, 2)]

### Step 2.2: Application

#### *helper function : generate_random_dag*

In [34]:
def generate_random_dag(num_nodes):
    dag = nx.DiGraph()

    nodes = [f"t{i+1}" for i in range(num_nodes)]
    dag.add_nodes_from(nodes)

    available_parents = {node: list(nodes[:i]) for i, node in enumerate(nodes)}

    for i in range(2, num_nodes + 1):
       
        num_parents = min(
            random.randint(1, min(i, max_num_parents_dag)), len(available_parents[f"t{i}"])
        )

        # select parents
        parent_nodes = random.sample(available_parents[f"t{i}"], num_parents)
        # add parents
        dag.add_edges_from((parent_node, f"t{i}") for parent_node in parent_nodes)

        # update available parents
        available_parents[f"t{i}"] = list(nodes[:i])

    return dag

#### *Generate task DAGs*

In [35]:
tasks_data = []

start_node_number = 1
for run in range(num_dag_generations):

    num_nodes = random.randint(min_num_nodes_dag, max_num_nodes_dag)

    random_dag = generate_random_dag(num_nodes)

    mapping = {
        f"t{i}": f"t{i + start_node_number - 1}" for i in range(1, num_nodes + 1)
    }

    random_dag = nx.relabel_nodes(random_dag, mapping)

    for node in random_dag.nodes:
        parents = list(random_dag.predecessors(node))
        task_info = {
            "id": node,
            "dependency": parents,
            "mobility": np.random.randint(1, 10),
            "kind": np.random.choice(task_kinds),
            "safe": np.random.choice([0, 1], p=[0.95, 0.05]),
            "computationalLoad": int(np.random.uniform(1, 100) * 1e4),
            "dataEntrySize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "returnDataSize": (np.random.randint(10, 100) // 10)
            * (10 ** np.random.choice([3, 6])),
            "status": "READY",
        }
        tasks_data.append(task_info)
    start_node_number += num_nodes

np.random.shuffle(tasks_data)
tasks = pd.DataFrame(tasks_data)

tasks.set_index("id", inplace=True)

tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status
id,,,,,,,,
t372,"[t367, t369, t370, t371]",5,3,1,958113,4000,1000000,READY
t46,"[t44, t45]",4,2,0,771795,8000,2000,READY
t843,[t840],6,1,1,475017,9000,9000,READY
t957,"[t954, t956]",8,3,0,241559,4000,8000000,READY
t814,"[t797, t802, t810]",5,2,0,461102,1000000,3000,READY
...,...,...,...,...,...,...,...,...
t49,"[t43, t44, t46, t47]",6,4,0,875949,2000,2000000,READY
t881,[],3,1,0,911683,2000,2000000,READY
t35,"[t32, t33, t34]",4,3,0,509560,6000000,6000,READY


## Step 3: Preprocessing

### Step 3.1: Clustering

#### *Tasks*

In [36]:
x = 0
for kind in task_kinds:
    for safe in (0, 1):
        selected_tasks = tasks.loc[(tasks["kind"] == kind) & (tasks["safe"] == safe), :]
        x += 1
        tasks.loc[selected_tasks.index, "cluster"] = x
        
tasks["cluster"] = tasks["cluster"].astype(int)
tasks

,dependency,mobility,kind,safe,computationalLoad,dataEntrySize,returnDataSize,status,cluster
id,,,,,,,,,
t372,"[t367, t369, t370, t371]",5,3,1,958113,4000,1000000,READY,6
t46,"[t44, t45]",4,2,0,771795,8000,2000,READY,3
t843,[t840],6,1,1,475017,9000,9000,READY,2
t957,"[t954, t956]",8,3,0,241559,4000,8000000,READY,5
t814,"[t797, t802, t810]",5,2,0,461102,1000000,3000,READY,3
...,...,...,...,...,...,...,...,...,...
t49,"[t43, t44, t46, t47]",6,4,0,875949,2000,2000000,READY,7
t881,[],3,1,0,911683,2000,2000000,READY,1
t35,"[t32, t33, t34]",4,3,0,509560,6000000,6000,READY,5


#### *IOT & MEC Devices*

In [37]:
x = 1
IoTdevices["cluster"] = [[] for _ in range(len(IoTdevices))]

for kind in task_kinds:
    for safe in (0, 1):
        selected_IoTdevices = IoTdevices[
            IoTdevices["accetableTasks"].apply(lambda lst: kind in lst)
        ]

        condition = (IoTdevices.index.isin(selected_IoTdevices.index)) & (
            (IoTdevices["handleSafeTask"] == 0) & (safe == 0)
            | (IoTdevices["handleSafeTask"] == 1)
        )

        IoTdevices.loc[condition, "cluster"] = IoTdevices.loc[
            condition, "cluster"
        ].apply(lambda lst: lst + [x])

        x += 1
# ---------------------------------
x = 1
MECDevices["cluster"] = [[] for _ in range(len(MECDevices))]

for kind in task_kinds:
    for safe in (0, 1):
        selected_MECDevices = MECDevices[
            MECDevices["accetableTasks"].apply(lambda lst: kind in lst)
        ]

        condition = (MECDevices.index.isin(selected_MECDevices.index)) & (
            (MECDevices["handleSafeTask"] == 0) & (safe == 0)
            | (MECDevices["handleSafeTask"] == 1)
        )

        MECDevices.loc[condition, "cluster"] = MECDevices.loc[
            condition, "cluster"
        ].apply(lambda lst: lst + [x])

        x += 1

MECDevices
IoTdevices

,number_of_cpu_cores,occupied_cores,voltages_frequencies,ISL,capacitance,powerIdle,batteryLevel,errorRate,accetableTasks,handleSafeTask,cluster
id,,,,,,,,,,,
0,8,"[0, 1, 1, 0, 0, 1, 1, 0]","[(1000000.0, 1.8), (8000000.0, 4.0), (32000000...",11,"[2.0335683668660422e-09, 2.6924918326971967e-0...","[0.0009, 0.0007, 0.0007999999999999999, 0.0009...",3.800000e+10,0.05,"[4, 1]",1,"[1, 2, 7, 8]"
1,4,"[0, 0, 1, 1]","[(16000000.0, 5.0), (2000000.0, 2.3), (3200000...",14,"[2.2224823345252286e-09, 2.5424163291696136e-0...","[0.0007, 0.0007999999999999999, 0.0009, 0.0007]",3.700000e+10,0.03,"[3, 1]",1,"[1, 2, 5, 6]"
2,4,"[0, 0, 0, 0]","[(32000000.0, 6.5), (1000000.0, 1.8), (1600000...",16,"[2.9431546396803817e-09, 2.7712209810637763e-0...","[0.0007999999999999999, 0.0007, 0.000799999999...",4.000000e+10,0.04,"[2, 1, 3]",1,"[1, 2, 3, 4, 5, 6]"
3,8,"[0, 1, 1, 0, 1, 0, 1, 0]","[(32000000.0, 6.5), (16000000.0, 5.0), (100000...",16,"[2.1513162264888043e-09, 2.1126715623024103e-0...","[0.0007999999999999999, 0.0007999999999999999,...",3.700000e+10,0.03,"[3, 1, 2]",1,"[1, 2, 3, 4, 5, 6]"
4,6,"[0, 1, 1, 1, 0, 1]","[(1000000.0, 1.8), (2000000.0, 2.3), (32000000...",19,"[2.3115550332130133e-09, 2.938119120978063e-09...","[0.0007, 0.0009, 0.0007999999999999999, 0.0009...",3.800000e+10,0.04,"[1, 2, 3]",1,"[1, 2, 3, 4, 5, 6]"


### Step 3.2: Queueing

In [38]:
sorted_tasks = tasks.sort_values(by="mobility").copy()
proccessingQueue = set()
for index, row in sorted_tasks.iterrows():
    if row["status"] == "Queued":
        continue
    for dep in row["dependency"]:
        if sorted_tasks.loc[dep, "status"] != "Queued":
            proccessingQueue.add(dep)
            sorted_tasks.loc[dep, "status"] = "Queued"
            dependecny_meet = False

    proccessingQueue.add(index)
    sorted_tasks.loc[index, "status"] = "Queued"

proccessingQueue = list(proccessingQueue)
proccessingQueue

['t422',
 't734',
 't274',
 't38',
 't1095',
 't428',
 't903',
 't1086',
 't699',
 't1037',
 't800',
 't390',
 't867',
 't701',
 't776',
 't349',
 't797',
 't204',
 't439',
 't807',
 't916',
 't777',
 't307',
 't363',
 't946',
 't519',
 't425',
 't11',
 't199',
 't243',
 't901',
 't557',
 't1092',
 't939',
 't427',
 't655',
 't405',
 't918',
 't959',
 't631',
 't445',
 't438',
 't82',
 't355',
 't741',
 't354',
 't936',
 't940',
 't366',
 't120',
 't460',
 't458',
 't594',
 't595',
 't1051',
 't386',
 't56',
 't347',
 't605',
 't794',
 't718',
 't1071',
 't324',
 't267',
 't1141',
 't658',
 't1057',
 't376',
 't231',
 't383',
 't724',
 't420',
 't509',
 't521',
 't786',
 't391',
 't728',
 't1101',
 't59',
 't856',
 't1121',
 't31',
 't852',
 't152',
 't752',
 't875',
 't938',
 't899',
 't632',
 't400',
 't1113',
 't626',
 't898',
 't45',
 't1094',
 't614',
 't335',
 't1091',
 't133',
 't591',
 't607',
 't477',
 't1031',
 't75',
 't820',
 't1024',
 't116',
 't397',
 't1049',
 't158',
 '

## Step 3.1: Setting Up The tree

In [39]:

class DDTNode(nn.Module):
    def __init__(self, feature_size, num_classes, depth, max_depth):
        """
        Initializes the DifferentiableDecisionTreeNode.

        Args:
        - feature_size (int): The size of the input features.
        - num_classes (int): The number of output classes.
        - depth (int): The depth of the current node in the decision tree.
        - max_depth (int): The maximum depth allowed for the decision tree.
        """
        super(DDTNode, self).__init__()
        self.feature_size = feature_size
        self.num_classes = num_classes
        self.depth = depth
        self.max_depth = max_depth
        # learnable parameters
        self.weight = nn.Parameter(torch.randn(feature_size))
        self.bias = nn.Parameter(torch.randn(1))
        self.prob_distribution = nn.Parameter(torch.zeros(num_classes))
        # If not at leaf, create left and right child nodes
        if self.depth < self.max_depth:
            self.left_child = DDTNode(feature_size, num_classes, depth+1, max_depth)
            self.right_child = DDTNode(feature_size, num_classes, depth+1, max_depth)

    def forward(self, x):
        """
        Forward pass of the DifferentiableDecisionTreeNode.

        Args:
        - x (torch.Tensor): Input data of shape (batch_size, feature_size).

        Returns:
        - torch.Tensor: Output probabilities for each class for each instance in the batch.
        """
        # Check if we are at a leaf node
        if self.depth == self.max_depth:
            # We are at a leaf, return the softmax probabilities for all instances in the batch
            probs = F.softmax(self.prob_distribution, dim=0)
            print(probs)
            return probs.expand(x.size(0), -1)  
        else:
            # Process each item in the batch individually (not efficient!) TODO make it vectorized
            decisions = torch.sigmoid(torch.matmul(x, self.weight) + self.bias)
            batch_results = torch.zeros(x.size(0), self.num_classes)
            for i in range(x.size(0)):
                decision = decisions[i]
                if decision > 0.5:
                    batch_results[i] = self.right_child(x[i:i+1])
                else:
                    batch_results[i] = self.left_child(x[i:i+1])
            return batch_results


class DDT(nn.Module):
    def __init__(self, feature_size, num_classes, max_depth):
        """
        Initializes the DDT (Differentiable Decision Tree) model.

        Args:
        - feature_size (int): The size of the input features.
        - num_classes (int): The number of output classes.
        - max_depth (int): The maximum depth allowed for the decision tree.
        """
        super(DDT, self).__init__()
        self.root = DDTNode(feature_size, num_classes, 0, max_depth)

    def forward(self, x):
        """
        Forward pass of the DDT model.

        Args:
        - x (torch.Tensor): Input data of shape (batch_size, feature_size).

        Returns:
        - torch.Tensor: Output probabilities for each class for each instance in the batch.
        """
        return self.root(x)
    


## Step 3.2: Custom data generation functions

In [40]:
def create_dataset(num_samples, num_features, num_classes):
    """
    Function to create a synthetic dataset.

    Args:
    - num_samples (int): Number of samples in the dataset.
    - num_features (int): Number of features for each sample.
    - num_classes (int): Number of classes for classification.

    Returns:
    - torch.Tensor: Features tensor of shape (num_samples, num_features).
    - torch.Tensor: Labels tensor of shape (num_samples,) containing class labels.
    """
    features = torch.rand(num_samples, num_features)  # Generate random features
    labels = torch.randint(0, num_classes, (num_samples,))  # Generate random class labels
    return features, labels


def create_imbalanced_dataset(num_samples, num_features, num_classes, focus_class):
    """
    Generate an imbalanced dataset with a focus on a specific class.

    Args:
        num_samples (int): Number of samples in the dataset.
        num_features (int): Number of features for each sample.
        num_classes (int): Number of classes for the labels.
        focus_class (int): The class to focus on.

    Returns:
        torch.Tensor: Features tensor of shape (num_samples, num_features).
        torch.Tensor: Labels tensor of shape (num_samples,).
    """
    # Generate random features
    features = torch.rand(num_samples, num_features)
    
    # Generate random labels
    labels = torch.randint(0, num_classes, (num_samples,))
    
    # Assign labels focusing on the desired class
    focus_indices = random.sample(range(num_samples), int(0.8 * num_samples))
    labels[focus_indices] = focus_class
    
    # Assign labels randomly to other classes
    other_classes = [c for c in range(num_classes) if c != focus_class]
    for idx in range(num_samples):
        if idx not in focus_indices:
            labels[idx] = random.choice(other_classes)
    
    # Shuffle the dataset
    indices = list(range(num_samples))
    random.shuffle(indices)
    features = features[indices]
    labels = labels[indices]
    
    return features, labels


## Step 3.3: Train & Test functions

In [41]:
# Define a function to train the model
def train(model, train_loader, criterion, optimizer, epochs):
    """
    Function to train the model.

    Args:
    - model (torch.nn.Module): The model to be trained.
    - train_loader (torch.utils.data.DataLoader): DataLoader for training dataset.
    - criterion (torch.nn.Module): Loss function.
    - optimizer (torch.optim.Optimizer): Optimization algorithm.
    - epochs (int): Number of epochs for training.
    """
    model.train()  # Set the model to training mode
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Clear gradients
            outputs = model(inputs.float())  # Forward pass
            loss = criterion(outputs, labels.long())  # Calculate loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update parameters
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Define a function to test the model
def test(model, test_loader, criterion):
    """
    Function to evaluate the model on the test set.

    Args:
    - model (torch.nn.Module): The trained model.
    - test_loader (torch.utils.data.DataLoader): DataLoader for test dataset.
    - criterion (torch.nn.Module): Loss function.
    """
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.float())
            test_loss += criterion(outputs, labels.long()).item()
            pred = outputs.argmax(dim=1, keepdim=True)
            correct += pred.eq(labels.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f"Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)")

# Generate the datasets
train_features, train_labels = create_dataset(800, 7, 3)
test_features, test_labels = create_dataset(200, 7, 3)

# Wrap the datasets in TensorDataset
train_dataset = TensorDataset(train_features, train_labels)
test_dataset = TensorDataset(test_features, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10)

# Initialize the model, loss function, and optimizer
model = DDT(7, 3, 3)  # Create a DDT model with input size 7, 3 classes, and maximum depth of 3
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Train and test the model
epochs = 10
train(model, train_loader, criterion, optimizer, epochs)  # Train the model
test(model, test_loader, criterion)  # Test the trained model


tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3330, 0.3337], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3330, 0.3337], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 

## Additional Step: Testing Probability Dist weights update process using a focused dataset

In [42]:
# Generate the imbalanced dataset
num_samples = 2000
num_features = 7
num_classes = 3
focus_class = 0

train_features, train_labels = create_imbalanced_dataset(num_samples, num_features, num_classes, focus_class)
test_features, test_labels = create_dataset(200, num_features, num_classes)
# test_features, test_labels = create_imbalanced_dataset(200, num_features, num_classes,focus_class)


# Wrap the datasets in TensorDataset
train_dataset = TensorDataset(train_features, train_labels)
test_dataset = TensorDataset(test_features, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10)

model = DDT(7, 3, 3)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Train and test the model
epochs = 10
train(model, train_loader, criterion, optimizer, epochs)
test(model, test_loader, criterion)

tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3333, 0.3333, 0.3333], grad_fn=<SoftmaxBackward0>)
tensor([0.3338, 0.3331, 0.3331], grad_fn=<SoftmaxBackward0>)
tensor([0.3338, 0.3331, 0.3331], grad_fn=<SoftmaxBackward0>)
tensor([0.3338, 0.3331, 0.3331], grad_fn=<SoftmaxBackward0>)
tensor([0.3338, 0.3331, 0.3331], grad_fn=<SoftmaxBackward0>)
tensor([0.3338, 0.3331, 0.3331], grad_fn=<SoftmaxBackward0>)
tensor([0.3338, 0.3331, 0.3331], grad_fn=<SoftmaxBackward0>)
tensor([0.3338, 0.3331, 

## Step 3.4: Predict Function

In [43]:
def predict(model, input_data):
    """
    Function to perform inference using the trained model.

    Args:
    - model (torch.nn.Module): The trained model.
    - input_data (torch.Tensor): Input data for prediction of shape (batch_size, num_features).

    Returns:
    - torch.Tensor: Predicted classes for each input instance.
    """
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        output = model(input_data.float())  # Forward pass
        predicted_classes = output.argmax(dim=1)  # Get the index of the maximum value (predicted class)
    return predicted_classes

# Example input data for prediction
input_data = torch.tensor([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7], [0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4]]) 

# Perform prediction using the trained model
predicted_classes = predict(model, input_data)

# Print the predicted classes
print("Predicted classes:", predicted_classes)


tensor([0.8813, 0.0596, 0.0591])
tensor([0.4744, 0.2727, 0.2529])
Predicted classes: tensor([0, 0])
